In [86]:
#!pip install catboost

In [1]:
import catboost as cb
import numpy as np
import pandas as pd
import seaborn as sns

from catboost import CatBoostRegressor

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [2]:
print(f"catBoost version: {cb.__version__}")
print(f"numpy version: {np.__version__}")
print(f"pandas version: {pd.__version__}")
print(f"seaborn version: {sns.__version__}")

catBoost version: 1.2.2
numpy version: 1.26.0
pandas version: 2.1.1
seaborn version: 0.12.2


In [3]:
print('Loading Properties ...')
properties2016 = pd.read_csv('properties_2016.csv', low_memory = False)
properties2017 = pd.read_csv('properties_2017.csv', low_memory = False)

print('Loading Train ...')
train2016 = pd.read_csv('train_2016_v2.csv', parse_dates=['transactiondate'], low_memory=False)
train2017 = pd.read_csv('train_2017.csv', parse_dates=['transactiondate'], low_memory=False)

print('Loading Sample ...')
sample_submission = pd.read_csv('sample_submission.csv', low_memory = False)

print('Done ...')

Loading Properties ...
Loading Train ...
Loading Sample ...
Done ...


---
## Exploring the csvs

In [41]:
print(train2016.shape)
train2016.head()

(90275, 5)


,parcelid,logerror,transaction_year,transaction_month,transaction_day
0,11016594,0.0276,2016,1,1
1,14366692,-0.1684,2016,1,1
2,12098116,-0.0040,2016,1,1
3,12643413,0.0218,2016,1,2
4,14432541,-0.0050,2016,1,2


In [5]:
print(train2017.shape)
train2017.head()

(77613, 3)


,parcelid,logerror,transactiondate
0,14297519,0.025595,1/1/2017
1,17052889,0.055619,1/1/2017
2,14186244,0.005383,1/1/2017
3,12177905,-0.103410,1/1/2017
4,10887214,0.006940,1/1/2017


In [6]:
print(properties2016.shape)
properties2016.head()

(2985217, 58)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,7.0,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [7]:
print(properties2017.shape)
properties2017.head()

(2985217, 58)


,parcelid,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,calculatedbathnbr,decktypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,9.0,2016.0,9.0,NaN,NaN,NaN,NaN
1,10759547,NaN,NaN,NaN,0.0,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,NaN,NaN,NaN,0.0,0.0,5.0,NaN,NaN,NaN,...,1.0,NaN,660680.0,1434941.0,2016.0,774261.0,20800.37,NaN,NaN,NaN
3,10859147,NaN,NaN,NaN,0.0,0.0,3.0,6.0,NaN,NaN,...,1.0,NaN,580059.0,1174475.0,2016.0,594416.0,14557.57,NaN,NaN,NaN
4,10879947,NaN,NaN,NaN,0.0,0.0,4.0,NaN,NaN,NaN,...,1.0,NaN,196751.0,440101.0,2016.0,243350.0,5725.17,NaN,NaN,NaN


In [8]:
print(sample_submission.shape)
sample_submission.head()

(2985217, 7)


,ParcelId,201610,201611,201612,201710,201711,201712
0,10754147,0,0,0,0,0,0
1,10759547,0,0,0,0,0,0
2,10843547,0,0,0,0,0,0
3,10859147,0,0,0,0,0,0
4,10879947,0,0,0,0,0,0


---
## Observing sample submission ParcelID order

In [9]:
sample_submission["ParcelId"].equals(properties2016["parcelid"])

True

#### The order of parcelids in the sample submission follows that off properties2016

## Creating test_df for final prediction and csv generation

In [10]:
test = pd.merge(sample_submission, properties2016, how='left', left_on ='ParcelId',right_on='parcelid')

In [11]:
test.head()

,ParcelId,201610,201611,201612,201710,201711,201712,parcelid,airconditioningtypeid,architecturalstyletypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,10754147,0,0,0,0,0,0,10754147,NaN,NaN,...,NaN,NaN,NaN,9.0,2015.0,9.0,NaN,NaN,NaN,NaN
1,10759547,0,0,0,0,0,0,10759547,NaN,NaN,...,NaN,NaN,NaN,27516.0,2015.0,27516.0,NaN,NaN,NaN,NaN
2,10843547,0,0,0,0,0,0,10843547,NaN,NaN,...,NaN,NaN,650756.0,1413387.0,2015.0,762631.0,20800.37,NaN,NaN,NaN
3,10859147,0,0,0,0,0,0,10859147,NaN,NaN,...,1.0,NaN,571346.0,1156834.0,2015.0,585488.0,14557.57,NaN,NaN,NaN
4,10879947,0,0,0,0,0,0,10879947,NaN,NaN,...,NaN,NaN,193796.0,433491.0,2015.0,239695.0,5725.17,NaN,NaN,NaN


In [12]:
test_columns = ['201610','201611','201612','201710','201711','201712']

---
## Splitting transaction dates into day, month, year

In [13]:
def split_date(df):
    df["transactiondate"] = pd.to_datetime(df["transactiondate"], dayfirst=True)
    df["transaction_year"] = df["transactiondate"].dt.year
    
    df["transaction_month"] = df["transactiondate"].dt.month
    
#     the code below is used if we want to let 2017 jan start as month 13 (might be useful when we combine both years to train a model)
#     df["transaction_month"] = (df["transactiondate"].dt.year - 2016)*12 + df["transactiondate"].dt.month

    
    df["transaction_day"] = df["transactiondate"].dt.day
    
    df.drop(["transactiondate"], inplace=True, axis=1)
    return df

train2016 = split_date(train2016)
train2017 = split_date(train2017)

In [14]:
print(train2016.shape)
train2016.head()

(90275, 5)


,parcelid,logerror,transaction_year,transaction_month,transaction_day
0,11016594,0.0276,2016,1,1
1,14366692,-0.1684,2016,1,1
2,12098116,-0.0040,2016,1,1
3,12643413,0.0218,2016,1,2
4,14432541,-0.0050,2016,1,2


In [15]:
print(train2017.shape)
train2017.head()

(77613, 5)


,parcelid,logerror,transaction_year,transaction_month,transaction_day
0,14297519,0.025595,2017,1,1
1,17052889,0.055619,2017,1,1
2,14186244,0.005383,2017,1,1
3,12177905,-0.103410,2017,1,1
4,10887214,0.006940,2017,1,1


In [16]:
train2016["parcelid"].nunique()

90150

In [17]:
train2017["parcelid"].nunique()

77414

#### Since the num unique values of parclid are less than the number of rows of train2016 and train2017, this means that there are houses that were resold --> something we have to handle later

## Non-Null counts for the columns in properties.csv

In [18]:
properties2016.info(show_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2985217 entries, 0 to 2985216
Data columns (total 58 columns):
 #   Column                        Non-Null Count    Dtype  
---  ------                        --------------    -----  
 0   parcelid                      2985217 non-null  int64  
 1   airconditioningtypeid         811519 non-null   float64
 2   architecturalstyletypeid      6061 non-null     float64
 3   basementsqft                  1628 non-null     float64
 4   bathroomcnt                   2973755 non-null  float64
 5   bedroomcnt                    2973767 non-null  float64
 6   buildingclasstypeid           12629 non-null    float64
 7   buildingqualitytypeid         1938488 non-null  float64
 8   calculatedbathnbr             2856305 non-null  float64
 9   decktypeid                    17096 non-null    float64
 10  finishedfloor1squarefeet      202717 non-null   float64
 11  calculatedfinishedsquarefeet  2929652 non-null  float64
 12  finishedsquarefeet12        

## Possible cleaning of dataframe is to remove the features with 1 unique value (no value added in a model)

(Not done yet)

## Labelling our numerical & categorical columns

In [19]:
nominal_features = ["censustractandblock",
                    "airconditioningtypeid",
                    "architecturalstyletypeid",
                    "buildingclasstypeid",
                    "decktypeid",
                    "fips",
                    "heatingorsystemtypeid",
                    "propertycountylandusecode",
                    "propertylandusetypeid",
                    "regionidcounty",
                    "storytypeid",
                    "typeconstructiontypeid",
                    "rawcensustractandblock",
                    "regionidneighborhood",
                    "propertyzoningdesc",
                    "regionidcity",
                    "regionidzip",
                    "hashottuborspa",
                    "fireplaceflag",
                    "pooltypeid10",
                    "pooltypeid2",
                    "pooltypeid7",
                    "taxdelinquencyflag"                    
                   ]

ordinal_features = ["bathroomcnt",
                    "bedroomcnt",
                    "buildingqualitytypeid",
                    "calculatedbathnbr",
                    "threequarterbathnbr",
                    "fireplacecnt",
                    "fullbathcnt",
                    "garagecarcnt",
                    "numberofstories",
                    "poolcnt",
                    "roomcnt",
                    "unitcnt",
                    "taxdelinquencyyear",
                    "yearbuilt",
                    "assessmentyear"] 

categorical_features = nominal_features + ordinal_features

numeric_features = ["basementsqft",
                    "finishedfloor1squarefeet",
                    "calculatedfinishedsquarefeet",
                    "finishedsquarefeet6",
                    "finishedsquarefeet12",
                    "finishedsquarefeet13",
                    "finishedsquarefeet15",
                    "finishedsquarefeet50",
                    "garagetotalsqft",
                    "latitude",
                    "longitude",
                    "lotsizesquarefeet",
                    "poolsizesum",
                    "yardbuildingsqft17",
                    "yardbuildingsqft26",
                    "taxvaluedollarcnt",
                    "structuretaxvaluedollarcnt",
                    "landtaxvaluedollarcnt",
                    "taxamount"]

In [20]:
print(f"numeric features length: {len(numeric_features)}")
print(f"categorical features length: {len(categorical_features)}")

numeric features length: 19
categorical features length: 38


## Preparing our train and test data

- our first approach is to combine both 2016 and 2017 data and train a model that predicts both years together
- we have to merge train.csv with their respective properties.csv


In [21]:
# how = left is equivalent to a left join in SQL, thus we will only have a dataframe with houses that are sold
combined2016 = pd.merge(train2016, properties2016, how = 'left', on = 'parcelid')  
combined2017 = pd.merge(train2017, properties2017, how = 'left', on = 'parcelid')

print(combined2016.shape)
print(combined2017.shape)

(90275, 62)
(77613, 62)


In [22]:
#we can see the merging of both train and properties
combined2016.head()

,parcelid,logerror,transaction_year,transaction_month,transaction_day,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.0276,2016,1,1,1.0,NaN,NaN,2.0,3.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.1684,2016,1,1,NaN,NaN,NaN,3.5,4.0,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.0040,2016,1,1,1.0,NaN,NaN,3.0,2.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.0218,2016,1,2,1.0,NaN,NaN,2.0,2.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.0050,2016,1,2,NaN,NaN,NaN,2.5,4.0,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13


In [23]:
# combining 2016 and 2017 for the final df
combined = pd.concat([combined2016, combined2017])
combined

,parcelid,logerror,transaction_year,transaction_month,transaction_day,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.027600,2016,1,1,1.0,NaN,NaN,2.0,3.0,...,NaN,NaN,122754.0,360170.0,2015.0,237416.0,6735.88,NaN,NaN,6.037107e+13
1,14366692,-0.168400,2016,1,1,NaN,NaN,NaN,3.5,4.0,...,NaN,NaN,346458.0,585529.0,2015.0,239071.0,10153.02,NaN,NaN,NaN
2,12098116,-0.004000,2016,1,1,1.0,NaN,NaN,3.0,2.0,...,NaN,NaN,61994.0,119906.0,2015.0,57912.0,11484.48,NaN,NaN,6.037464e+13
3,12643413,0.021800,2016,1,2,1.0,NaN,NaN,2.0,2.0,...,NaN,NaN,171518.0,244880.0,2015.0,73362.0,3048.74,NaN,NaN,6.037296e+13
4,14432541,-0.005000,2016,1,2,NaN,NaN,NaN,2.5,4.0,...,2.0,NaN,169574.0,434551.0,2015.0,264977.0,5488.96,NaN,NaN,6.059042e+13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,10833991,-0.002245,2017,9,20,1.0,NaN,NaN,3.0,3.0,...,NaN,NaN,265000.0,379000.0,2016.0,114000.0,4685.34,NaN,NaN,6.037132e+13
77609,11000655,0.020615,2017,9,20,NaN,NaN,NaN,2.0,2.0,...,NaN,NaN,70917.0,354621.0,2016.0,283704.0,4478.43,NaN,NaN,6.037101e+13
77610,17239384,0.013209,2017,9,21,NaN,NaN,NaN,2.0,4.0,...,1.0,NaN,50683.0,67205.0,2016.0,16522.0,1107.48,NaN,NaN,6.111008e+13
77611,12773139,0.037129,2017,9,21,1.0,NaN,NaN,1.0,3.0,...,NaN,NaN,32797.0,49546.0,2016.0,16749.0,876.43,NaN,NaN,6.037434e+13


## Populating NaN values 

We first try filling them up with empty strings

In [24]:
combined.fillna('', inplace=True)

C:\Users\Darren Choo\AppData\Local\Temp\ipykernel_20464\3665335949.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  combined.fillna('', inplace=True)


In [25]:
combined

,parcelid,logerror,transaction_year,transaction_month,transaction_day,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,11016594,0.027600,2016,1,1,1.0,,,2.0,3.0,...,,,122754.0,360170.0,2015.0,237416.0,6735.88,,,60371066461001.0
1,14366692,-0.168400,2016,1,1,,,,3.5,4.0,...,,,346458.0,585529.0,2015.0,239071.0,10153.02,,,
2,12098116,-0.004000,2016,1,1,1.0,,,3.0,2.0,...,,,61994.0,119906.0,2015.0,57912.0,11484.48,,,60374638003004.0
3,12643413,0.021800,2016,1,2,1.0,,,2.0,2.0,...,,,171518.0,244880.0,2015.0,73362.0,3048.74,,,60372963002002.0
4,14432541,-0.005000,2016,1,2,,,,2.5,4.0,...,2.0,,169574.0,434551.0,2015.0,264977.0,5488.96,,,60590423381006.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77608,10833991,-0.002245,2017,9,20,1.0,,,3.0,3.0,...,,,265000.0,379000.0,2016.0,114000.0,4685.34,,,60371319001006.0
77609,11000655,0.020615,2017,9,20,,,,2.0,2.0,...,,,70917.0,354621.0,2016.0,283704.0,4478.43,,,60371014003002.0
77610,17239384,0.013209,2017,9,21,,,,2.0,4.0,...,1.0,,50683.0,67205.0,2016.0,16522.0,1107.48,,,61110084022016.0
77611,12773139,0.037129,2017,9,21,1.0,,,1.0,3.0,...,,,32797.0,49546.0,2016.0,16749.0,876.43,,,60374338022005.0


## Converting all categorical feature values to strings (a requirement for catBoost)

- either this, or i might have to use jinruis method of label encoding

In [26]:
type(combined["transaction_year"].iloc[0])

numpy.int32

In [27]:
#since we have split the transactiondate into year, month, day, we will have to add them to the categorical features
categorical_features += ["transaction_year", "transaction_month", "transaction_day"]
len(categorical_features)

41

In [28]:
for i in categorical_features:
    combined[i] = combined[i].astype(str)

In [29]:
type(combined["transaction_year"].iloc[0])

str

## Getting our train / test splits


In [30]:
y = combined["logerror"]
X = combined.drop(["parcelid", "logerror"], axis=1)

In [31]:
print(X.shape)
X.head()

(167888, 60)


,transaction_year,transaction_month,transaction_day,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
0,2016,1,1,1.0,,,2.0,3.0,,4.0,...,,,122754.0,360170.0,2015.0,237416.0,6735.88,,,60371066461001.0
1,2016,1,1,,,,3.5,4.0,,,...,,,346458.0,585529.0,2015.0,239071.0,10153.02,,,
2,2016,1,1,1.0,,,3.0,2.0,,4.0,...,,,61994.0,119906.0,2015.0,57912.0,11484.48,,,60374638003004.0
3,2016,1,2,1.0,,,2.0,2.0,,4.0,...,,,171518.0,244880.0,2015.0,73362.0,3048.74,,,60372963002002.0
4,2016,1,2,,,,2.5,4.0,,,...,2.0,,169574.0,434551.0,2015.0,264977.0,5488.96,,,60590423381006.0


In [32]:
y.head()

0    0.0276
1   -0.1684
2   -0.0040
3    0.0218
4   -0.0050
Name: logerror, dtype: float64

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1)

print(f"X_train size: {len(X_train)}")
print(f"X_test size: {len(X_test)}")
print(f"y_train size: {len(y_train)}")
print(f"y_test size: {len(y_test)}")

X_train size: 134310
X_test size: 33578
y_train size: 134310
y_test size: 33578


In [34]:
X_train.head()

,transaction_year,transaction_month,transaction_day,airconditioningtypeid,architecturalstyletypeid,basementsqft,bathroomcnt,bedroomcnt,buildingclasstypeid,buildingqualitytypeid,...,numberofstories,fireplaceflag,structuretaxvaluedollarcnt,taxvaluedollarcnt,assessmentyear,landtaxvaluedollarcnt,taxamount,taxdelinquencyflag,taxdelinquencyyear,censustractandblock
1145,2016,1,7,,,,3.0,2.0,,1.0,...,,,254938.0,410564.0,2015.0,155626.0,5008.74,,,60372641031011.0
46858,2017,6,15,,,,2.0,3.0,,6.0,...,,,35960.0,108540.0,2016.0,72580.0,1353.25,,,60371411021004.0
26074,2017,4,13,,,,2.5,3.0,,,...,2.0,,256156.0,512312.0,2016.0,256156.0,5692.0,,,61110076141139.0
43052,2016,6,7,,,,2.0,3.0,,4.0,...,,,97580.0,166558.0,2015.0,68978.0,2174.84,,,60372141004000.0
76357,2016,9,15,,,,1.0,1.0,,,...,2.0,,10547.0,410000.0,2015.0,399453.0,4320.64,,,60590320111002.0


In [35]:
y_train.head()

1145     0.045000
46858   -2.271428
26074    0.008852
43052   -0.098700
76357    0.035400
Name: logerror, dtype: float64

## Before we train a catBoost model, we need to get indices of categorical columns from X_train

In [36]:
categorical_indices = []
for i in categorical_features:
    idx = X_train.columns.get_loc(i)
    categorical_indices.append(idx)

categorical_indices.sort()
       
print(categorical_indices)

[0, 1, 2, 3, 4, 6, 7, 8, 9, 10, 11, 19, 20, 21, 22, 24, 25, 29, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 49, 50, 51, 54, 57, 58, 59]


# Trying out best params from online kaggle model

In [37]:
catboost_regressor = CatBoostRegressor(learning_rate = 0.03, 
                                       depth = 6, 
                                       loss_function = 'MAE',
                                       eval_metric = 'MAE',
                                       n_estimators = 900,
                                       l2_leaf_reg = 3,
                                       cat_features = categorical_indices
                                      )

catboost_regressor.fit(X_train, y_train)

0:	learn: 0.0688215	total: 680ms	remaining: 10m 11s
1:	learn: 0.0688030	total: 1.13s	remaining: 8m 26s
2:	learn: 0.0687841	total: 1.52s	remaining: 7m 35s
3:	learn: 0.0687675	total: 1.92s	remaining: 7m 10s
4:	learn: 0.0687515	total: 2.34s	remaining: 6m 58s
5:	learn: 0.0687365	total: 2.73s	remaining: 6m 46s
6:	learn: 0.0687207	total: 3.14s	remaining: 6m 41s
7:	learn: 0.0687053	total: 3.55s	remaining: 6m 35s
8:	learn: 0.0686936	total: 3.95s	remaining: 6m 31s
9:	learn: 0.0686803	total: 4.35s	remaining: 6m 27s
10:	learn: 0.0686681	total: 4.79s	remaining: 6m 27s
11:	learn: 0.0686558	total: 5.2s	remaining: 6m 25s
12:	learn: 0.0686432	total: 5.64s	remaining: 6m 25s
13:	learn: 0.0686329	total: 6.09s	remaining: 6m 25s
14:	learn: 0.0686208	total: 6.54s	remaining: 6m 25s
15:	learn: 0.0686128	total: 6.94s	remaining: 6m 23s
16:	learn: 0.0686045	total: 7.36s	remaining: 6m 22s
17:	learn: 0.0685964	total: 7.79s	remaining: 6m 21s
18:	learn: 0.0685866	total: 8.2s	remaining: 6m 19s
19:	learn: 0.0685771	to

158:	learn: 0.0680602	total: 1m 7s	remaining: 5m 15s
159:	learn: 0.0680572	total: 1m 8s	remaining: 5m 15s
160:	learn: 0.0680556	total: 1m 8s	remaining: 5m 14s
161:	learn: 0.0680537	total: 1m 8s	remaining: 5m 14s
162:	learn: 0.0680519	total: 1m 9s	remaining: 5m 13s
163:	learn: 0.0680493	total: 1m 9s	remaining: 5m 13s
164:	learn: 0.0680480	total: 1m 10s	remaining: 5m 12s
165:	learn: 0.0680466	total: 1m 10s	remaining: 5m 11s
166:	learn: 0.0680451	total: 1m 10s	remaining: 5m 11s
167:	learn: 0.0680435	total: 1m 11s	remaining: 5m 10s
168:	learn: 0.0680419	total: 1m 11s	remaining: 5m 10s
169:	learn: 0.0680404	total: 1m 12s	remaining: 5m 9s
170:	learn: 0.0680388	total: 1m 12s	remaining: 5m 9s
171:	learn: 0.0680371	total: 1m 13s	remaining: 5m 9s
172:	learn: 0.0680351	total: 1m 13s	remaining: 5m 8s
173:	learn: 0.0680342	total: 1m 13s	remaining: 5m 7s
174:	learn: 0.0680333	total: 1m 14s	remaining: 5m 7s
175:	learn: 0.0680300	total: 1m 14s	remaining: 5m 6s
176:	learn: 0.0680285	total: 1m 14s	remai

311:	learn: 0.0678697	total: 2m 12s	remaining: 4m 10s
312:	learn: 0.0678687	total: 2m 13s	remaining: 4m 9s
313:	learn: 0.0678681	total: 2m 13s	remaining: 4m 9s
314:	learn: 0.0678665	total: 2m 14s	remaining: 4m 8s
315:	learn: 0.0678653	total: 2m 14s	remaining: 4m 8s
316:	learn: 0.0678649	total: 2m 14s	remaining: 4m 7s
317:	learn: 0.0678644	total: 2m 15s	remaining: 4m 7s
318:	learn: 0.0678635	total: 2m 15s	remaining: 4m 7s
319:	learn: 0.0678624	total: 2m 16s	remaining: 4m 6s
320:	learn: 0.0678614	total: 2m 16s	remaining: 4m 6s
321:	learn: 0.0678607	total: 2m 16s	remaining: 4m 5s
322:	learn: 0.0678598	total: 2m 17s	remaining: 4m 5s
323:	learn: 0.0678596	total: 2m 17s	remaining: 4m 5s
324:	learn: 0.0678588	total: 2m 18s	remaining: 4m 5s
325:	learn: 0.0678576	total: 2m 18s	remaining: 4m 4s
326:	learn: 0.0678559	total: 2m 19s	remaining: 4m 4s
327:	learn: 0.0678529	total: 2m 19s	remaining: 4m 3s
328:	learn: 0.0678517	total: 2m 20s	remaining: 4m 3s
329:	learn: 0.0678506	total: 2m 20s	remaining

464:	learn: 0.0677003	total: 3m 20s	remaining: 3m 7s
465:	learn: 0.0676988	total: 3m 20s	remaining: 3m 7s
466:	learn: 0.0676975	total: 3m 21s	remaining: 3m 6s
467:	learn: 0.0676951	total: 3m 21s	remaining: 3m 6s
468:	learn: 0.0676939	total: 3m 22s	remaining: 3m 5s
469:	learn: 0.0676931	total: 3m 22s	remaining: 3m 5s
470:	learn: 0.0676910	total: 3m 23s	remaining: 3m 4s
471:	learn: 0.0676901	total: 3m 23s	remaining: 3m 4s
472:	learn: 0.0676894	total: 3m 23s	remaining: 3m 4s
473:	learn: 0.0676884	total: 3m 24s	remaining: 3m 3s
474:	learn: 0.0676860	total: 3m 24s	remaining: 3m 3s
475:	learn: 0.0676853	total: 3m 25s	remaining: 3m 2s
476:	learn: 0.0676839	total: 3m 25s	remaining: 3m 2s
477:	learn: 0.0676829	total: 3m 26s	remaining: 3m 1s
478:	learn: 0.0676817	total: 3m 26s	remaining: 3m 1s
479:	learn: 0.0676798	total: 3m 26s	remaining: 3m 1s
480:	learn: 0.0676786	total: 3m 27s	remaining: 3m
481:	learn: 0.0676776	total: 3m 28s	remaining: 3m
482:	learn: 0.0676765	total: 3m 28s	remaining: 2m 59

618:	learn: 0.0675167	total: 4m 29s	remaining: 2m 2s
619:	learn: 0.0675159	total: 4m 29s	remaining: 2m 1s
620:	learn: 0.0675152	total: 4m 30s	remaining: 2m 1s
621:	learn: 0.0675142	total: 4m 30s	remaining: 2m
622:	learn: 0.0675126	total: 4m 31s	remaining: 2m
623:	learn: 0.0675115	total: 4m 31s	remaining: 2m
624:	learn: 0.0675096	total: 4m 32s	remaining: 1m 59s
625:	learn: 0.0675081	total: 4m 32s	remaining: 1m 59s
626:	learn: 0.0675072	total: 4m 33s	remaining: 1m 58s
627:	learn: 0.0675061	total: 4m 33s	remaining: 1m 58s
628:	learn: 0.0675040	total: 4m 33s	remaining: 1m 58s
629:	learn: 0.0675027	total: 4m 34s	remaining: 1m 57s
630:	learn: 0.0675023	total: 4m 34s	remaining: 1m 57s
631:	learn: 0.0675004	total: 4m 35s	remaining: 1m 56s
632:	learn: 0.0675001	total: 4m 35s	remaining: 1m 56s
633:	learn: 0.0674995	total: 4m 36s	remaining: 1m 55s
634:	learn: 0.0674992	total: 4m 36s	remaining: 1m 55s
635:	learn: 0.0674972	total: 4m 37s	remaining: 1m 55s
636:	learn: 0.0674965	total: 4m 37s	remaini

772:	learn: 0.0673788	total: 5m 46s	remaining: 57s
773:	learn: 0.0673780	total: 5m 47s	remaining: 56.6s
774:	learn: 0.0673775	total: 5m 48s	remaining: 56.1s
775:	learn: 0.0673771	total: 5m 48s	remaining: 55.7s
776:	learn: 0.0673763	total: 5m 49s	remaining: 55.3s
777:	learn: 0.0673760	total: 5m 49s	remaining: 54.8s
778:	learn: 0.0673753	total: 5m 49s	remaining: 54.4s
779:	learn: 0.0673740	total: 5m 50s	remaining: 53.9s
780:	learn: 0.0673732	total: 5m 50s	remaining: 53.5s
781:	learn: 0.0673719	total: 5m 51s	remaining: 53s
782:	learn: 0.0673713	total: 5m 51s	remaining: 52.6s
783:	learn: 0.0673707	total: 5m 52s	remaining: 52.1s
784:	learn: 0.0673703	total: 5m 52s	remaining: 51.7s
785:	learn: 0.0673700	total: 5m 53s	remaining: 51.2s
786:	learn: 0.0673696	total: 5m 53s	remaining: 50.8s
787:	learn: 0.0673675	total: 5m 54s	remaining: 50.4s
788:	learn: 0.0673671	total: 5m 54s	remaining: 49.9s
789:	learn: 0.0673658	total: 5m 55s	remaining: 49.5s
790:	learn: 0.0673655	total: 5m 55s	remaining: 49s

In [38]:
pred = catboost_regressor.predict(X_test)

#calculating MAE
print("MAE: ", mean_absolute_error(y_test, pred))

#calculating R-squared
print("R-squared: ", r2_score(y_test, pred))

MAE:  0.06803908698292017
R-squared:  0.00730741500100196


In [39]:
pred.shape

(33578,)

## Creating our catBoost model & Predicting

#### 500 iterations (loss function: RMSE)

In [42]:
catboost_regressor = CatBoostRegressor(iterations = 500,
                                       learning_rate = 0.001, 
                                       depth = 6, 
                                       loss_function = 'RMSE',
                                       cat_features = categorical_indices,
                                      )

catboost_regressor.fit(X_train, y_train)

0:	learn: 0.1667179	total: 414ms	remaining: 3m 26s
1:	learn: 0.1667171	total: 789ms	remaining: 3m 16s
2:	learn: 0.1667166	total: 1.16s	remaining: 3m 11s
3:	learn: 0.1667155	total: 1.57s	remaining: 3m 14s
4:	learn: 0.1667155	total: 1.67s	remaining: 2m 44s
5:	learn: 0.1667148	total: 2.04s	remaining: 2m 48s
6:	learn: 0.1667138	total: 2.41s	remaining: 2m 49s
7:	learn: 0.1667138	total: 2.47s	remaining: 2m 31s
8:	learn: 0.1667137	total: 2.81s	remaining: 2m 33s
9:	learn: 0.1667127	total: 3.17s	remaining: 2m 35s
10:	learn: 0.1667117	total: 3.52s	remaining: 2m 36s
11:	learn: 0.1667106	total: 3.88s	remaining: 2m 37s
12:	learn: 0.1667095	total: 4.23s	remaining: 2m 38s
13:	learn: 0.1667085	total: 4.6s	remaining: 2m 39s
14:	learn: 0.1667082	total: 4.96s	remaining: 2m 40s
15:	learn: 0.1667076	total: 5.34s	remaining: 2m 41s
16:	learn: 0.1667066	total: 5.71s	remaining: 2m 42s
17:	learn: 0.1667060	total: 6.09s	remaining: 2m 43s
18:	learn: 0.1667056	total: 6.46s	remaining: 2m 43s
19:	learn: 0.1667042	to

159:	learn: 0.1666142	total: 55.3s	remaining: 1m 57s
160:	learn: 0.1666142	total: 55.5s	remaining: 1m 56s
161:	learn: 0.1666142	total: 55.6s	remaining: 1m 56s
162:	learn: 0.1666130	total: 56s	remaining: 1m 55s
163:	learn: 0.1666129	total: 56.1s	remaining: 1m 54s
164:	learn: 0.1666124	total: 56.5s	remaining: 1m 54s
165:	learn: 0.1666113	total: 56.9s	remaining: 1m 54s
166:	learn: 0.1666107	total: 57.3s	remaining: 1m 54s
167:	learn: 0.1666099	total: 57.7s	remaining: 1m 53s
168:	learn: 0.1666099	total: 58.1s	remaining: 1m 53s
169:	learn: 0.1666099	total: 58.3s	remaining: 1m 53s
170:	learn: 0.1666088	total: 58.6s	remaining: 1m 52s
171:	learn: 0.1666079	total: 58.9s	remaining: 1m 52s
172:	learn: 0.1666074	total: 59.4s	remaining: 1m 52s
173:	learn: 0.1666074	total: 59.5s	remaining: 1m 51s
174:	learn: 0.1666063	total: 59.9s	remaining: 1m 51s
175:	learn: 0.1666063	total: 1m	remaining: 1m 50s
176:	learn: 0.1666053	total: 1m	remaining: 1m 50s
177:	learn: 0.1666046	total: 1m 1s	remaining: 1m 50s
1

312:	learn: 0.1665160	total: 1m 54s	remaining: 1m 8s
313:	learn: 0.1665149	total: 1m 54s	remaining: 1m 7s
314:	learn: 0.1665139	total: 1m 54s	remaining: 1m 7s
315:	learn: 0.1665133	total: 1m 55s	remaining: 1m 7s
316:	learn: 0.1665133	total: 1m 55s	remaining: 1m 6s
317:	learn: 0.1665123	total: 1m 56s	remaining: 1m 6s
318:	learn: 0.1665112	total: 1m 56s	remaining: 1m 6s
319:	learn: 0.1665112	total: 1m 56s	remaining: 1m 5s
320:	learn: 0.1665104	total: 1m 56s	remaining: 1m 5s
321:	learn: 0.1665097	total: 1m 57s	remaining: 1m 4s
322:	learn: 0.1665090	total: 1m 57s	remaining: 1m 4s
323:	learn: 0.1665081	total: 1m 58s	remaining: 1m 4s
324:	learn: 0.1665078	total: 1m 58s	remaining: 1m 3s
325:	learn: 0.1665070	total: 1m 59s	remaining: 1m 3s
326:	learn: 0.1665063	total: 1m 59s	remaining: 1m 3s
327:	learn: 0.1665057	total: 1m 59s	remaining: 1m 2s
328:	learn: 0.1665047	total: 2m	remaining: 1m 2s
329:	learn: 0.1665039	total: 2m	remaining: 1m 2s
330:	learn: 0.1665028	total: 2m 1s	remaining: 1m 1s
33

469:	learn: 0.1664119	total: 3m 9s	remaining: 12.1s
470:	learn: 0.1664113	total: 3m 10s	remaining: 11.7s
471:	learn: 0.1664107	total: 3m 10s	remaining: 11.3s
472:	learn: 0.1664100	total: 3m 11s	remaining: 10.9s
473:	learn: 0.1664091	total: 3m 11s	remaining: 10.5s
474:	learn: 0.1664085	total: 3m 12s	remaining: 10.1s
475:	learn: 0.1664077	total: 3m 12s	remaining: 9.73s
476:	learn: 0.1664068	total: 3m 13s	remaining: 9.33s
477:	learn: 0.1664058	total: 3m 14s	remaining: 8.94s
478:	learn: 0.1664047	total: 3m 14s	remaining: 8.54s
479:	learn: 0.1664047	total: 3m 15s	remaining: 8.13s
480:	learn: 0.1664038	total: 3m 15s	remaining: 7.73s
481:	learn: 0.1664037	total: 3m 16s	remaining: 7.33s
482:	learn: 0.1664031	total: 3m 16s	remaining: 6.92s
483:	learn: 0.1664029	total: 3m 17s	remaining: 6.52s
484:	learn: 0.1664029	total: 3m 17s	remaining: 6.11s
485:	learn: 0.1664023	total: 3m 18s	remaining: 5.71s
486:	learn: 0.1664014	total: 3m 18s	remaining: 5.3s
487:	learn: 0.1664007	total: 3m 19s	remaining: 4

In [43]:
pred = catboost_regressor.predict(X_test)

#calculating MAE
print("MAE: ", mean_absolute_error(y_test, pred))

#calculating R-squared
print("R-squared: ", r2_score(y_test, pred))

MAE:  0.06942320110830578
R-squared:  0.001428379127954349


#### 500 iterations (loss function: MAE)

In [44]:
catboost_regressor = CatBoostRegressor(iterations = 500,
                                       learning_rate = 0.001, 
                                       depth = 6, 
                                       loss_function = 'MAE',
                                       cat_features = categorical_indices,
                                      )
catboost_regressor.fit(X_train, y_train)

0:	learn: 0.0688385	total: 690ms	remaining: 5m 44s
1:	learn: 0.0688379	total: 1.32s	remaining: 5m 29s
2:	learn: 0.0688374	total: 1.89s	remaining: 5m 12s
3:	learn: 0.0688368	total: 2.48s	remaining: 5m 6s
4:	learn: 0.0688362	total: 3.07s	remaining: 5m 3s
5:	learn: 0.0688355	total: 3.7s	remaining: 5m 4s
6:	learn: 0.0688349	total: 4.27s	remaining: 5m
7:	learn: 0.0688342	total: 4.9s	remaining: 5m 1s
8:	learn: 0.0688336	total: 5.49s	remaining: 4m 59s
9:	learn: 0.0688331	total: 6.04s	remaining: 4m 56s
10:	learn: 0.0688324	total: 6.71s	remaining: 4m 58s
11:	learn: 0.0688319	total: 7.29s	remaining: 4m 56s
12:	learn: 0.0688313	total: 7.92s	remaining: 4m 56s
13:	learn: 0.0688307	total: 8.76s	remaining: 5m 4s
14:	learn: 0.0688302	total: 9.31s	remaining: 5m
15:	learn: 0.0688296	total: 9.91s	remaining: 4m 59s
16:	learn: 0.0688290	total: 10.5s	remaining: 4m 58s
17:	learn: 0.0688283	total: 11.1s	remaining: 4m 56s
18:	learn: 0.0688277	total: 11.7s	remaining: 4m 55s
19:	learn: 0.0688271	total: 12.3s	rem

158:	learn: 0.0687499	total: 1m 30s	remaining: 3m 14s
159:	learn: 0.0687494	total: 1m 31s	remaining: 3m 13s
160:	learn: 0.0687489	total: 1m 31s	remaining: 3m 13s
161:	learn: 0.0687483	total: 1m 32s	remaining: 3m 12s
162:	learn: 0.0687478	total: 1m 33s	remaining: 3m 12s
163:	learn: 0.0687473	total: 1m 33s	remaining: 3m 11s
164:	learn: 0.0687468	total: 1m 34s	remaining: 3m 11s
165:	learn: 0.0687463	total: 1m 34s	remaining: 3m 10s
166:	learn: 0.0687458	total: 1m 35s	remaining: 3m 9s
167:	learn: 0.0687453	total: 1m 35s	remaining: 3m 9s
168:	learn: 0.0687449	total: 1m 36s	remaining: 3m 8s
169:	learn: 0.0687443	total: 1m 37s	remaining: 3m 8s
170:	learn: 0.0687438	total: 1m 37s	remaining: 3m 7s
171:	learn: 0.0687433	total: 1m 37s	remaining: 3m 6s
172:	learn: 0.0687429	total: 1m 38s	remaining: 3m 6s
173:	learn: 0.0687424	total: 1m 39s	remaining: 3m 5s
174:	learn: 0.0687419	total: 1m 39s	remaining: 3m 5s
175:	learn: 0.0687414	total: 1m 40s	remaining: 3m 4s
176:	learn: 0.0687409	total: 1m 40s	re

312:	learn: 0.0686797	total: 2m 49s	remaining: 1m 41s
313:	learn: 0.0686793	total: 2m 49s	remaining: 1m 40s
314:	learn: 0.0686789	total: 2m 50s	remaining: 1m 39s
315:	learn: 0.0686785	total: 2m 50s	remaining: 1m 39s
316:	learn: 0.0686781	total: 2m 51s	remaining: 1m 38s
317:	learn: 0.0686777	total: 2m 51s	remaining: 1m 38s
318:	learn: 0.0686773	total: 2m 52s	remaining: 1m 37s
319:	learn: 0.0686770	total: 2m 52s	remaining: 1m 37s
320:	learn: 0.0686766	total: 2m 53s	remaining: 1m 36s
321:	learn: 0.0686762	total: 2m 54s	remaining: 1m 36s
322:	learn: 0.0686757	total: 2m 54s	remaining: 1m 35s
323:	learn: 0.0686753	total: 2m 55s	remaining: 1m 35s
324:	learn: 0.0686749	total: 2m 55s	remaining: 1m 34s
325:	learn: 0.0686745	total: 2m 56s	remaining: 1m 34s
326:	learn: 0.0686741	total: 2m 56s	remaining: 1m 33s
327:	learn: 0.0686737	total: 2m 57s	remaining: 1m 33s
328:	learn: 0.0686733	total: 2m 57s	remaining: 1m 32s
329:	learn: 0.0686729	total: 2m 58s	remaining: 1m 31s
330:	learn: 0.0686725	total:

467:	learn: 0.0686193	total: 4m 14s	remaining: 17.4s
468:	learn: 0.0686189	total: 4m 14s	remaining: 16.8s
469:	learn: 0.0686186	total: 4m 15s	remaining: 16.3s
470:	learn: 0.0686182	total: 4m 15s	remaining: 15.7s
471:	learn: 0.0686179	total: 4m 16s	remaining: 15.2s
472:	learn: 0.0686175	total: 4m 16s	remaining: 14.7s
473:	learn: 0.0686171	total: 4m 17s	remaining: 14.1s
474:	learn: 0.0686168	total: 4m 17s	remaining: 13.6s
475:	learn: 0.0686164	total: 4m 18s	remaining: 13s
476:	learn: 0.0686161	total: 4m 18s	remaining: 12.5s
477:	learn: 0.0686157	total: 4m 19s	remaining: 11.9s
478:	learn: 0.0686153	total: 4m 20s	remaining: 11.4s
479:	learn: 0.0686148	total: 4m 20s	remaining: 10.9s
480:	learn: 0.0686144	total: 4m 21s	remaining: 10.3s
481:	learn: 0.0686141	total: 4m 22s	remaining: 9.79s
482:	learn: 0.0686138	total: 4m 22s	remaining: 9.24s
483:	learn: 0.0686135	total: 4m 23s	remaining: 8.69s
484:	learn: 0.0686131	total: 4m 23s	remaining: 8.15s
485:	learn: 0.0686128	total: 4m 23s	remaining: 7

In [45]:
pred = catboost_regressor.predict(X_test)

#calculating MAE
print("MAE: ", mean_absolute_error(y_test, pred))

#calculating R-squared
print("R-squared: ", r2_score(y_test, pred))

MAE:  0.06871884329007173
R-squared:  -1.1829909201566124e-05


#### 1000 iterations (loss function: RMSE)

In [46]:
catboost_regressor = CatBoostRegressor(iterations = 1000,
                                       learning_rate = 0.001, 
                                       depth = 6, 
                                       loss_function = 'RMSE',
                                       cat_features = categorical_indices,
                                      )
catboost_regressor.fit(X_train, y_train)

0:	learn: 0.1667179	total: 510ms	remaining: 8m 29s
1:	learn: 0.1667171	total: 1s	remaining: 8m 20s
2:	learn: 0.1667166	total: 1.49s	remaining: 8m 15s
3:	learn: 0.1667155	total: 1.99s	remaining: 8m 14s
4:	learn: 0.1667155	total: 2.08s	remaining: 6m 54s
5:	learn: 0.1667148	total: 2.57s	remaining: 7m 5s
6:	learn: 0.1667138	total: 3.06s	remaining: 7m 14s
7:	learn: 0.1667138	total: 3.15s	remaining: 6m 30s
8:	learn: 0.1667137	total: 3.63s	remaining: 6m 39s
9:	learn: 0.1667127	total: 4.12s	remaining: 6m 47s
10:	learn: 0.1667117	total: 4.66s	remaining: 6m 59s
11:	learn: 0.1667106	total: 5.11s	remaining: 7m 1s
12:	learn: 0.1667095	total: 5.53s	remaining: 6m 59s
13:	learn: 0.1667085	total: 5.97s	remaining: 7m
14:	learn: 0.1667082	total: 6.39s	remaining: 6m 59s
15:	learn: 0.1667076	total: 6.82s	remaining: 6m 59s
16:	learn: 0.1667066	total: 7.25s	remaining: 6m 58s
17:	learn: 0.1667060	total: 7.7s	remaining: 7m
18:	learn: 0.1667056	total: 8.12s	remaining: 6m 59s
19:	learn: 0.1667042	total: 8.55s	re

159:	learn: 0.1666142	total: 1m 7s	remaining: 5m 56s
160:	learn: 0.1666142	total: 1m 8s	remaining: 5m 54s
161:	learn: 0.1666142	total: 1m 8s	remaining: 5m 52s
162:	learn: 0.1666130	total: 1m 8s	remaining: 5m 52s
163:	learn: 0.1666129	total: 1m 8s	remaining: 5m 50s
164:	learn: 0.1666124	total: 1m 9s	remaining: 5m 50s
165:	learn: 0.1666113	total: 1m 9s	remaining: 5m 49s
166:	learn: 0.1666107	total: 1m 10s	remaining: 5m 49s
167:	learn: 0.1666099	total: 1m 10s	remaining: 5m 49s
168:	learn: 0.1666099	total: 1m 10s	remaining: 5m 49s
169:	learn: 0.1666099	total: 1m 11s	remaining: 5m 47s
170:	learn: 0.1666088	total: 1m 11s	remaining: 5m 46s
171:	learn: 0.1666079	total: 1m 11s	remaining: 5m 46s
172:	learn: 0.1666074	total: 1m 12s	remaining: 5m 46s
173:	learn: 0.1666074	total: 1m 12s	remaining: 5m 44s
174:	learn: 0.1666063	total: 1m 12s	remaining: 5m 43s
175:	learn: 0.1666063	total: 1m 13s	remaining: 5m 43s
176:	learn: 0.1666053	total: 1m 13s	remaining: 5m 43s
177:	learn: 0.1666046	total: 1m 14s

312:	learn: 0.1665160	total: 2m 10s	remaining: 4m 45s
313:	learn: 0.1665149	total: 2m 10s	remaining: 4m 45s
314:	learn: 0.1665139	total: 2m 11s	remaining: 4m 45s
315:	learn: 0.1665133	total: 2m 11s	remaining: 4m 44s
316:	learn: 0.1665133	total: 2m 11s	remaining: 4m 43s
317:	learn: 0.1665123	total: 2m 12s	remaining: 4m 43s
318:	learn: 0.1665112	total: 2m 12s	remaining: 4m 43s
319:	learn: 0.1665112	total: 2m 12s	remaining: 4m 42s
320:	learn: 0.1665104	total: 2m 13s	remaining: 4m 41s
321:	learn: 0.1665097	total: 2m 13s	remaining: 4m 41s
322:	learn: 0.1665090	total: 2m 14s	remaining: 4m 41s
323:	learn: 0.1665081	total: 2m 14s	remaining: 4m 40s
324:	learn: 0.1665078	total: 2m 15s	remaining: 4m 40s
325:	learn: 0.1665070	total: 2m 15s	remaining: 4m 40s
326:	learn: 0.1665063	total: 2m 16s	remaining: 4m 39s
327:	learn: 0.1665057	total: 2m 16s	remaining: 4m 40s
328:	learn: 0.1665047	total: 2m 17s	remaining: 4m 39s
329:	learn: 0.1665039	total: 2m 17s	remaining: 4m 39s
330:	learn: 0.1665028	total:

465:	learn: 0.1664145	total: 3m 20s	remaining: 3m 50s
466:	learn: 0.1664142	total: 3m 21s	remaining: 3m 49s
467:	learn: 0.1664134	total: 3m 22s	remaining: 3m 49s
468:	learn: 0.1664127	total: 3m 22s	remaining: 3m 49s
469:	learn: 0.1664119	total: 3m 23s	remaining: 3m 49s
470:	learn: 0.1664113	total: 3m 23s	remaining: 3m 48s
471:	learn: 0.1664107	total: 3m 24s	remaining: 3m 48s
472:	learn: 0.1664100	total: 3m 24s	remaining: 3m 48s
473:	learn: 0.1664091	total: 3m 25s	remaining: 3m 47s
474:	learn: 0.1664085	total: 3m 25s	remaining: 3m 47s
475:	learn: 0.1664077	total: 3m 26s	remaining: 3m 47s
476:	learn: 0.1664068	total: 3m 26s	remaining: 3m 46s
477:	learn: 0.1664058	total: 3m 27s	remaining: 3m 46s
478:	learn: 0.1664047	total: 3m 28s	remaining: 3m 46s
479:	learn: 0.1664047	total: 3m 28s	remaining: 3m 46s
480:	learn: 0.1664038	total: 3m 29s	remaining: 3m 45s
481:	learn: 0.1664037	total: 3m 29s	remaining: 3m 45s
482:	learn: 0.1664031	total: 3m 30s	remaining: 3m 45s
483:	learn: 0.1664029	total:

618:	learn: 0.1663252	total: 4m 38s	remaining: 2m 51s
619:	learn: 0.1663245	total: 4m 38s	remaining: 2m 50s
620:	learn: 0.1663239	total: 4m 39s	remaining: 2m 50s
621:	learn: 0.1663238	total: 4m 39s	remaining: 2m 49s
622:	learn: 0.1663232	total: 4m 40s	remaining: 2m 49s
623:	learn: 0.1663223	total: 4m 40s	remaining: 2m 49s
624:	learn: 0.1663223	total: 4m 41s	remaining: 2m 48s
625:	learn: 0.1663219	total: 4m 41s	remaining: 2m 48s
626:	learn: 0.1663215	total: 4m 42s	remaining: 2m 47s
627:	learn: 0.1663212	total: 4m 42s	remaining: 2m 47s
628:	learn: 0.1663206	total: 4m 43s	remaining: 2m 47s
629:	learn: 0.1663204	total: 4m 43s	remaining: 2m 46s
630:	learn: 0.1663193	total: 4m 44s	remaining: 2m 46s
631:	learn: 0.1663183	total: 4m 45s	remaining: 2m 45s
632:	learn: 0.1663173	total: 4m 45s	remaining: 2m 45s
633:	learn: 0.1663166	total: 4m 46s	remaining: 2m 45s
634:	learn: 0.1663161	total: 4m 46s	remaining: 2m 44s
635:	learn: 0.1663157	total: 4m 47s	remaining: 2m 44s
636:	learn: 0.1663153	total:

771:	learn: 0.1662482	total: 5m 59s	remaining: 1m 46s
772:	learn: 0.1662477	total: 5m 59s	remaining: 1m 45s
773:	learn: 0.1662476	total: 6m	remaining: 1m 45s
774:	learn: 0.1662470	total: 6m	remaining: 1m 44s
775:	learn: 0.1662465	total: 6m 1s	remaining: 1m 44s
776:	learn: 0.1662461	total: 6m 1s	remaining: 1m 43s
777:	learn: 0.1662451	total: 6m 2s	remaining: 1m 43s
778:	learn: 0.1662442	total: 6m 2s	remaining: 1m 42s
779:	learn: 0.1662438	total: 6m 3s	remaining: 1m 42s
780:	learn: 0.1662428	total: 6m 3s	remaining: 1m 42s
781:	learn: 0.1662425	total: 6m 4s	remaining: 1m 41s
782:	learn: 0.1662417	total: 6m 5s	remaining: 1m 41s
783:	learn: 0.1662416	total: 6m 5s	remaining: 1m 40s
784:	learn: 0.1662412	total: 6m 6s	remaining: 1m 40s
785:	learn: 0.1662405	total: 6m 7s	remaining: 1m 39s
786:	learn: 0.1662398	total: 6m 7s	remaining: 1m 39s
787:	learn: 0.1662393	total: 6m 8s	remaining: 1m 39s
788:	learn: 0.1662393	total: 6m 8s	remaining: 1m 38s
789:	learn: 0.1662384	total: 6m 9s	remaining: 1m 3

926:	learn: 0.1661759	total: 7m 16s	remaining: 34.4s
927:	learn: 0.1661758	total: 7m 16s	remaining: 33.9s
928:	learn: 0.1661750	total: 7m 17s	remaining: 33.4s
929:	learn: 0.1661748	total: 7m 17s	remaining: 33s
930:	learn: 0.1661747	total: 7m 18s	remaining: 32.5s
931:	learn: 0.1661747	total: 7m 18s	remaining: 32s
932:	learn: 0.1661742	total: 7m 19s	remaining: 31.6s
933:	learn: 0.1661735	total: 7m 19s	remaining: 31.1s
934:	learn: 0.1661731	total: 7m 20s	remaining: 30.6s
935:	learn: 0.1661721	total: 7m 21s	remaining: 30.2s
936:	learn: 0.1661717	total: 7m 21s	remaining: 29.7s
937:	learn: 0.1661713	total: 7m 22s	remaining: 29.3s
938:	learn: 0.1661707	total: 7m 23s	remaining: 28.8s
939:	learn: 0.1661705	total: 7m 23s	remaining: 28.3s
940:	learn: 0.1661697	total: 7m 24s	remaining: 27.9s
941:	learn: 0.1661697	total: 7m 24s	remaining: 27.4s
942:	learn: 0.1661691	total: 7m 25s	remaining: 26.9s
943:	learn: 0.1661687	total: 7m 25s	remaining: 26.5s
944:	learn: 0.1661682	total: 7m 26s	remaining: 26s

In [47]:
pred = catboost_regressor.predict(X_test)

#calculating MAE
print("MAE: ", mean_absolute_error(y_test, pred))

#calculating R-squared
print("R-squared: ", r2_score(y_test, pred))

MAE:  0.06932158557363569
R-squared:  0.0024107445661211147


#### 1000 iterations (loss function: MAE)

In [50]:
catboost_regressor = CatBoostRegressor(iterations = 1000,
                                       learning_rate = 0.001, 
                                       depth = 6, 
                                       loss_function = 'MAE',
                                       cat_features = categorical_indices,
                                      )
catboost_regressor.fit(X_train, y_train)

0:	learn: 0.0688385	total: 491ms	remaining: 8m 10s
1:	learn: 0.0688379	total: 967ms	remaining: 8m 2s
2:	learn: 0.0688374	total: 1.37s	remaining: 7m 34s
3:	learn: 0.0688368	total: 1.8s	remaining: 7m 27s
4:	learn: 0.0688362	total: 2.23s	remaining: 7m 22s
5:	learn: 0.0688355	total: 2.68s	remaining: 7m 24s
6:	learn: 0.0688349	total: 3.08s	remaining: 7m 17s
7:	learn: 0.0688342	total: 3.53s	remaining: 7m 17s
8:	learn: 0.0688336	total: 3.96s	remaining: 7m 15s
9:	learn: 0.0688331	total: 4.37s	remaining: 7m 13s
10:	learn: 0.0688324	total: 4.79s	remaining: 7m 11s
11:	learn: 0.0688319	total: 5.21s	remaining: 7m 8s
12:	learn: 0.0688313	total: 5.64s	remaining: 7m 8s
13:	learn: 0.0688307	total: 6.09s	remaining: 7m 9s
14:	learn: 0.0688302	total: 6.53s	remaining: 7m 8s
15:	learn: 0.0688296	total: 6.98s	remaining: 7m 9s
16:	learn: 0.0688290	total: 7.38s	remaining: 7m 6s
17:	learn: 0.0688283	total: 7.81s	remaining: 7m 5s
18:	learn: 0.0688277	total: 8.26s	remaining: 7m 6s
19:	learn: 0.0688271	total: 8.67

158:	learn: 0.0687499	total: 1m 9s	remaining: 6m 8s
159:	learn: 0.0687494	total: 1m 10s	remaining: 6m 7s
160:	learn: 0.0687489	total: 1m 10s	remaining: 6m 7s
161:	learn: 0.0687483	total: 1m 10s	remaining: 6m 6s
162:	learn: 0.0687478	total: 1m 11s	remaining: 6m 6s
163:	learn: 0.0687473	total: 1m 11s	remaining: 6m 6s
164:	learn: 0.0687468	total: 1m 12s	remaining: 6m 5s
165:	learn: 0.0687463	total: 1m 12s	remaining: 6m 5s
166:	learn: 0.0687458	total: 1m 13s	remaining: 6m 4s
167:	learn: 0.0687453	total: 1m 13s	remaining: 6m 4s
168:	learn: 0.0687449	total: 1m 14s	remaining: 6m 4s
169:	learn: 0.0687443	total: 1m 14s	remaining: 6m 3s
170:	learn: 0.0687438	total: 1m 14s	remaining: 6m 2s
171:	learn: 0.0687433	total: 1m 15s	remaining: 6m 2s
172:	learn: 0.0687429	total: 1m 15s	remaining: 6m 2s
173:	learn: 0.0687424	total: 1m 16s	remaining: 6m 1s
174:	learn: 0.0687419	total: 1m 16s	remaining: 6m 1s
175:	learn: 0.0687414	total: 1m 17s	remaining: 6m 1s
176:	learn: 0.0687409	total: 1m 17s	remaining: 

311:	learn: 0.0686802	total: 2m 22s	remaining: 5m 13s
312:	learn: 0.0686797	total: 2m 22s	remaining: 5m 13s
313:	learn: 0.0686793	total: 2m 23s	remaining: 5m 12s
314:	learn: 0.0686789	total: 2m 23s	remaining: 5m 12s
315:	learn: 0.0686785	total: 2m 24s	remaining: 5m 11s
316:	learn: 0.0686781	total: 2m 24s	remaining: 5m 11s
317:	learn: 0.0686777	total: 2m 24s	remaining: 5m 10s
318:	learn: 0.0686773	total: 2m 25s	remaining: 5m 10s
319:	learn: 0.0686770	total: 2m 25s	remaining: 5m 9s
320:	learn: 0.0686766	total: 2m 26s	remaining: 5m 9s
321:	learn: 0.0686762	total: 2m 26s	remaining: 5m 8s
322:	learn: 0.0686757	total: 2m 27s	remaining: 5m 8s
323:	learn: 0.0686753	total: 2m 27s	remaining: 5m 7s
324:	learn: 0.0686749	total: 2m 28s	remaining: 5m 7s
325:	learn: 0.0686745	total: 2m 28s	remaining: 5m 7s
326:	learn: 0.0686741	total: 2m 29s	remaining: 5m 6s
327:	learn: 0.0686737	total: 2m 29s	remaining: 5m 6s
328:	learn: 0.0686733	total: 2m 30s	remaining: 5m 6s
329:	learn: 0.0686729	total: 2m 30s	re

464:	learn: 0.0686204	total: 3m 40s	remaining: 4m 13s
465:	learn: 0.0686200	total: 3m 40s	remaining: 4m 12s
466:	learn: 0.0686197	total: 3m 41s	remaining: 4m 12s
467:	learn: 0.0686193	total: 3m 41s	remaining: 4m 12s
468:	learn: 0.0686189	total: 3m 42s	remaining: 4m 11s
469:	learn: 0.0686186	total: 3m 42s	remaining: 4m 11s
470:	learn: 0.0686182	total: 3m 43s	remaining: 4m 10s
471:	learn: 0.0686179	total: 3m 43s	remaining: 4m 10s
472:	learn: 0.0686175	total: 3m 44s	remaining: 4m 9s
473:	learn: 0.0686171	total: 3m 44s	remaining: 4m 9s
474:	learn: 0.0686168	total: 3m 45s	remaining: 4m 9s
475:	learn: 0.0686164	total: 3m 45s	remaining: 4m 8s
476:	learn: 0.0686161	total: 3m 46s	remaining: 4m 8s
477:	learn: 0.0686157	total: 3m 46s	remaining: 4m 7s
478:	learn: 0.0686153	total: 3m 47s	remaining: 4m 7s
479:	learn: 0.0686148	total: 3m 47s	remaining: 4m 6s
480:	learn: 0.0686144	total: 3m 48s	remaining: 4m 6s
481:	learn: 0.0686141	total: 3m 49s	remaining: 4m 6s
482:	learn: 0.0686138	total: 3m 49s	re

618:	learn: 0.0685686	total: 5m	remaining: 3m 4s
619:	learn: 0.0685683	total: 5m	remaining: 3m 4s
620:	learn: 0.0685680	total: 5m 1s	remaining: 3m 3s
621:	learn: 0.0685678	total: 5m 1s	remaining: 3m 3s
622:	learn: 0.0685675	total: 5m 2s	remaining: 3m 2s
623:	learn: 0.0685671	total: 5m 2s	remaining: 3m 2s
624:	learn: 0.0685668	total: 5m 3s	remaining: 3m 1s
625:	learn: 0.0685665	total: 5m 3s	remaining: 3m 1s
626:	learn: 0.0685661	total: 5m 3s	remaining: 3m
627:	learn: 0.0685658	total: 5m 4s	remaining: 3m
628:	learn: 0.0685655	total: 5m 4s	remaining: 2m 59s
629:	learn: 0.0685653	total: 5m 5s	remaining: 2m 59s
630:	learn: 0.0685650	total: 5m 5s	remaining: 2m 58s
631:	learn: 0.0685646	total: 5m 6s	remaining: 2m 58s
632:	learn: 0.0685643	total: 5m 6s	remaining: 2m 57s
633:	learn: 0.0685640	total: 5m 7s	remaining: 2m 57s
634:	learn: 0.0685637	total: 5m 7s	remaining: 2m 56s
635:	learn: 0.0685634	total: 5m 8s	remaining: 2m 56s
636:	learn: 0.0685631	total: 5m 8s	remaining: 2m 55s
637:	learn: 0.0

772:	learn: 0.0685236	total: 6m 30s	remaining: 1m 54s
773:	learn: 0.0685233	total: 6m 31s	remaining: 1m 54s
774:	learn: 0.0685230	total: 6m 32s	remaining: 1m 53s
775:	learn: 0.0685227	total: 6m 32s	remaining: 1m 53s
776:	learn: 0.0685224	total: 6m 33s	remaining: 1m 52s
777:	learn: 0.0685222	total: 6m 33s	remaining: 1m 52s
778:	learn: 0.0685219	total: 6m 34s	remaining: 1m 51s
779:	learn: 0.0685216	total: 6m 34s	remaining: 1m 51s
780:	learn: 0.0685213	total: 6m 35s	remaining: 1m 50s
781:	learn: 0.0685211	total: 6m 35s	remaining: 1m 50s
782:	learn: 0.0685208	total: 6m 36s	remaining: 1m 49s
783:	learn: 0.0685205	total: 6m 37s	remaining: 1m 49s
784:	learn: 0.0685203	total: 6m 38s	remaining: 1m 49s
785:	learn: 0.0685201	total: 6m 38s	remaining: 1m 48s
786:	learn: 0.0685198	total: 6m 39s	remaining: 1m 48s
787:	learn: 0.0685195	total: 6m 40s	remaining: 1m 47s
788:	learn: 0.0685192	total: 6m 41s	remaining: 1m 47s
789:	learn: 0.0685190	total: 6m 41s	remaining: 1m 46s
790:	learn: 0.0685188	total:

926:	learn: 0.0684834	total: 8m 7s	remaining: 38.4s
927:	learn: 0.0684832	total: 8m 7s	remaining: 37.8s
928:	learn: 0.0684830	total: 8m 8s	remaining: 37.3s
929:	learn: 0.0684828	total: 8m 8s	remaining: 36.8s
930:	learn: 0.0684826	total: 8m 9s	remaining: 36.3s
931:	learn: 0.0684823	total: 8m 9s	remaining: 35.7s
932:	learn: 0.0684821	total: 8m 10s	remaining: 35.2s
933:	learn: 0.0684818	total: 8m 10s	remaining: 34.7s
934:	learn: 0.0684815	total: 8m 11s	remaining: 34.2s
935:	learn: 0.0684813	total: 8m 11s	remaining: 33.6s
936:	learn: 0.0684810	total: 8m 12s	remaining: 33.1s
937:	learn: 0.0684807	total: 8m 13s	remaining: 32.6s
938:	learn: 0.0684805	total: 8m 13s	remaining: 32.1s
939:	learn: 0.0684803	total: 8m 14s	remaining: 31.5s
940:	learn: 0.0684800	total: 8m 14s	remaining: 31s
941:	learn: 0.0684797	total: 8m 15s	remaining: 30.5s
942:	learn: 0.0684795	total: 8m 15s	remaining: 30s
943:	learn: 0.0684792	total: 8m 16s	remaining: 29.4s
944:	learn: 0.0684789	total: 8m 16s	remaining: 28.9s
945

In [51]:
pred = catboost_regressor.predict(X_test)

#calculating MAE
print("MAE: ", mean_absolute_error(y_test, pred))

#calculating R-squared
print("R-squared: ", r2_score(y_test, pred))

MAE:  0.0685891515983562
R-squared:  0.0009329146839222036


#### 1000 iterations (loss function: RMSE) with L2 leaf regularization (3)

In [ ]:
catboost_regressor = CatBoostRegressor(iterations = 1000,
                                       learning_rate = 0.001, 
                                       depth = 6, 
                                       loss_function = 'RMSE',
                                       cat_features = categorical_indices,
                                       l2_leaf_reg=3
                                      )
catboost_regressor.fit(X_train, y_train)

In [ ]:
pred = catboost_regressor.predict(X_test)

#calculating MAE
print("MAE: ", mean_absolute_error(y_test, pred))

#calculating R-squared
print("R-squared: ", r2_score(y_test, pred))

#### 1000 iterations (loss function: MAE) with L2 leaf regularization

#### 5000 iterations

# Trying out catBoost gridsearch 1st set of params


First with loss function RMSE

In [55]:
model = CatBoostRegressor(loss_function = 'RMSE', cat_features = categorical_indices)

In [58]:
grid = {'iterations': [100, 150, 200],
        'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

model.grid_search(grid, 
                  X=X_train, 
                  y=y_train,
                  cv=3)

0:	learn: 0.1662452	test: 0.1713872	best: 0.1713872 (0)	total: 77.3ms	remaining: 7.65s
1:	learn: 0.1662055	test: 0.1713507	best: 0.1713507 (1)	total: 144ms	remaining: 7.05s
2:	learn: 0.1661612	test: 0.1713112	best: 0.1713112 (2)	total: 183ms	remaining: 5.92s
3:	learn: 0.1661248	test: 0.1712778	best: 0.1712778 (3)	total: 227ms	remaining: 5.45s
4:	learn: 0.1660898	test: 0.1712463	best: 0.1712463 (4)	total: 289ms	remaining: 5.49s
5:	learn: 0.1660555	test: 0.1712151	best: 0.1712151 (5)	total: 329ms	remaining: 5.15s
6:	learn: 0.1660218	test: 0.1711841	best: 0.1711841 (6)	total: 363ms	remaining: 4.82s
7:	learn: 0.1659909	test: 0.1711560	best: 0.1711560 (7)	total: 399ms	remaining: 4.58s
8:	learn: 0.1659632	test: 0.1711313	best: 0.1711313 (8)	total: 436ms	remaining: 4.41s
9:	learn: 0.1659343	test: 0.1711067	best: 0.1711067 (9)	total: 471ms	remaining: 4.24s
10:	learn: 0.1659023	test: 0.1710778	best: 0.1710778 (10)	total: 512ms	remaining: 4.14s
11:	learn: 0.1658803	test: 0.1710578	best: 0.171057

99:	learn: 0.1651905	test: 0.1704560	best: 0.1704560 (99)	total: 3.9s	remaining: 0us

bestTest = 0.1704559994
bestIteration = 99

0:	loss: 0.1704560	best: 0.1704560 (0)	total: 4.2s	remaining: 6m 38s
0:	learn: 0.1661290	test: 0.1712830	best: 0.1712830 (0)	total: 67.9ms	remaining: 6.72s
1:	learn: 0.1660183	test: 0.1711817	best: 0.1711817 (1)	total: 125ms	remaining: 6.11s
2:	learn: 0.1659082	test: 0.1710856	best: 0.1710856 (2)	total: 164ms	remaining: 5.29s
3:	learn: 0.1658295	test: 0.1710145	best: 0.1710145 (3)	total: 203ms	remaining: 4.88s
4:	learn: 0.1657472	test: 0.1709448	best: 0.1709448 (4)	total: 261ms	remaining: 4.96s
5:	learn: 0.1656915	test: 0.1708957	best: 0.1708957 (5)	total: 300ms	remaining: 4.7s
6:	learn: 0.1656373	test: 0.1708431	best: 0.1708431 (6)	total: 339ms	remaining: 4.5s
7:	learn: 0.1656001	test: 0.1708073	best: 0.1708073 (7)	total: 379ms	remaining: 4.36s
8:	learn: 0.1655672	test: 0.1707844	best: 0.1707844 (8)	total: 426ms	remaining: 4.3s
9:	learn: 0.1655332	test: 0.1

92:	learn: 0.1648257	test: 0.1702061	best: 0.1702061 (92)	total: 3.52s	remaining: 265ms
93:	learn: 0.1648147	test: 0.1701959	best: 0.1701959 (93)	total: 3.56s	remaining: 227ms
94:	learn: 0.1648133	test: 0.1701959	best: 0.1701959 (93)	total: 3.59s	remaining: 189ms
95:	learn: 0.1648123	test: 0.1701945	best: 0.1701945 (95)	total: 3.63s	remaining: 151ms
96:	learn: 0.1648091	test: 0.1701946	best: 0.1701945 (95)	total: 3.67s	remaining: 113ms
97:	learn: 0.1648079	test: 0.1701943	best: 0.1701943 (97)	total: 3.71s	remaining: 75.6ms
98:	learn: 0.1648068	test: 0.1701933	best: 0.1701933 (98)	total: 3.74s	remaining: 37.8ms
99:	learn: 0.1647938	test: 0.1701671	best: 0.1701671 (99)	total: 3.78s	remaining: 0us

bestTest = 0.1701670505
bestIteration = 99

1:	loss: 0.1701671	best: 0.1701671 (1)	total: 8.02s	remaining: 6m 17s
0:	learn: 0.1662452	test: 0.1713872	best: 0.1713872 (0)	total: 67.4ms	remaining: 6.67s
1:	learn: 0.1662055	test: 0.1713507	best: 0.1713507 (1)	total: 136ms	remaining: 6.65s
2:	learn

89:	learn: 0.1652422	test: 0.1704949	best: 0.1704949 (89)	total: 3.47s	remaining: 386ms
90:	learn: 0.1652410	test: 0.1704949	best: 0.1704949 (89)	total: 3.51s	remaining: 347ms
91:	learn: 0.1652344	test: 0.1704902	best: 0.1704902 (91)	total: 3.54s	remaining: 308ms
92:	learn: 0.1652330	test: 0.1704891	best: 0.1704891 (92)	total: 3.58s	remaining: 269ms
93:	learn: 0.1652318	test: 0.1704884	best: 0.1704884 (93)	total: 3.61s	remaining: 231ms
94:	learn: 0.1652283	test: 0.1704881	best: 0.1704881 (94)	total: 3.65s	remaining: 192ms
95:	learn: 0.1652235	test: 0.1704839	best: 0.1704839 (95)	total: 3.69s	remaining: 154ms
96:	learn: 0.1652212	test: 0.1704820	best: 0.1704820 (96)	total: 3.72s	remaining: 115ms
97:	learn: 0.1652176	test: 0.1704787	best: 0.1704787 (97)	total: 3.76s	remaining: 76.8ms
98:	learn: 0.1652154	test: 0.1704799	best: 0.1704787 (97)	total: 3.8s	remaining: 38.4ms
99:	learn: 0.1652130	test: 0.1704775	best: 0.1704775 (99)	total: 3.83s	remaining: 0us

bestTest = 0.1704775335
bestIter

83:	learn: 0.1648797	test: 0.1702943	best: 0.1702943 (83)	total: 3.2s	remaining: 609ms
84:	learn: 0.1648777	test: 0.1702930	best: 0.1702930 (84)	total: 3.24s	remaining: 571ms
85:	learn: 0.1648771	test: 0.1702925	best: 0.1702925 (85)	total: 3.27s	remaining: 533ms
86:	learn: 0.1648744	test: 0.1702925	best: 0.1702925 (86)	total: 3.31s	remaining: 495ms
87:	learn: 0.1648724	test: 0.1702888	best: 0.1702888 (87)	total: 3.35s	remaining: 456ms
88:	learn: 0.1648667	test: 0.1702843	best: 0.1702843 (88)	total: 3.38s	remaining: 418ms
89:	learn: 0.1648607	test: 0.1702747	best: 0.1702747 (89)	total: 3.42s	remaining: 380ms
90:	learn: 0.1648590	test: 0.1702751	best: 0.1702747 (89)	total: 3.46s	remaining: 342ms
91:	learn: 0.1648563	test: 0.1702729	best: 0.1702729 (91)	total: 3.5s	remaining: 305ms
92:	learn: 0.1648425	test: 0.1702668	best: 0.1702668 (92)	total: 3.54s	remaining: 267ms
93:	learn: 0.1648341	test: 0.1702579	best: 0.1702579 (93)	total: 3.58s	remaining: 228ms
94:	learn: 0.1648328	test: 0.17025

...

124:	learn: 0.1634633	test: 0.1616404	best: 0.1616176 (119)	total: 18.9s	remaining: 3.78s
125:	learn: 0.1634351	test: 0.1616378	best: 0.1616176 (119)	total: 19s	remaining: 3.63s
126:	learn: 0.1633607	test: 0.1616387	best: 0.1616176 (119)	total: 19.2s	remaining: 3.48s
127:	learn: 0.1632731	test: 0.1616317	best: 0.1616176 (119)	total: 19.4s	remaining: 3.33s
128:	learn: 0.1632295	test: 0.1616327	best: 0.1616176 (119)	total: 19.5s	remaining: 3.17s
129:	learn: 0.1631965	test: 0.1616432	best: 0.1616176 (119)	total: 19.7s	remaining: 3.02s
130:	learn: 0.1631602	test: 0.1616336	best: 0.1616176 (119)	total: 19.8s	remaining: 2.87s
131:	learn: 0.1631257	test: 0.1616296	best: 0.1616176 (119)	total: 20s	remaining: 2.72s
132:	learn: 0.1630722	test: 0.1616327	best: 0.1616176 (119)	total: 20.1s	remaining: 2.57s
133:	learn: 0.1630502	test: 0.1616372	best: 0.1616176 (119)	total: 20.3s	remaining: 2.42s
134:	learn: 0.1630116	test: 0.1616495	best: 0.1616176 (119)	total: 20.5s	remaining: 2.27s
135:	learn: 0.

68:	learn: 0.1636473	test: 0.1647582	best: 0.1647576 (67)	total: 10.4s	remaining: 12.2s
69:	learn: 0.1636025	test: 0.1647495	best: 0.1647495 (69)	total: 10.5s	remaining: 12s
70:	learn: 0.1635152	test: 0.1647546	best: 0.1647495 (69)	total: 10.7s	remaining: 11.9s
71:	learn: 0.1635148	test: 0.1647520	best: 0.1647495 (69)	total: 10.7s	remaining: 11.6s
72:	learn: 0.1634956	test: 0.1647472	best: 0.1647472 (72)	total: 10.9s	remaining: 11.5s
73:	learn: 0.1634907	test: 0.1647453	best: 0.1647453 (73)	total: 11s	remaining: 11.3s
74:	learn: 0.1634457	test: 0.1647298	best: 0.1647298 (74)	total: 11.2s	remaining: 11.2s
75:	learn: 0.1633977	test: 0.1647118	best: 0.1647118 (75)	total: 11.4s	remaining: 11.1s
76:	learn: 0.1633717	test: 0.1647133	best: 0.1647118 (75)	total: 11.5s	remaining: 10.9s
77:	learn: 0.1633233	test: 0.1647002	best: 0.1647002 (77)	total: 11.7s	remaining: 10.8s
78:	learn: 0.1633007	test: 0.1646942	best: 0.1646942 (78)	total: 11.8s	remaining: 10.6s
79:	learn: 0.1632814	test: 0.1646890

10:	learn: 0.1632921	test: 0.1714583	best: 0.1714583 (10)	total: 1.8s	remaining: 22.8s
11:	learn: 0.1632446	test: 0.1714452	best: 0.1714452 (11)	total: 1.96s	remaining: 22.6s
12:	learn: 0.1632161	test: 0.1714226	best: 0.1714226 (12)	total: 2.12s	remaining: 22.3s
13:	learn: 0.1632003	test: 0.1714118	best: 0.1714118 (13)	total: 2.27s	remaining: 22.1s
14:	learn: 0.1631463	test: 0.1713913	best: 0.1713913 (14)	total: 2.43s	remaining: 21.9s
15:	learn: 0.1630608	test: 0.1713854	best: 0.1713854 (15)	total: 2.58s	remaining: 21.6s
16:	learn: 0.1629400	test: 0.1713762	best: 0.1713762 (16)	total: 2.74s	remaining: 21.4s
17:	learn: 0.1628665	test: 0.1713644	best: 0.1713644 (17)	total: 2.9s	remaining: 21.3s
18:	learn: 0.1628537	test: 0.1713507	best: 0.1713507 (18)	total: 3.04s	remaining: 21s
19:	learn: 0.1628167	test: 0.1713246	best: 0.1713246 (19)	total: 3.19s	remaining: 20.7s
20:	learn: 0.1627448	test: 0.1713117	best: 0.1713117 (20)	total: 3.35s	remaining: 20.6s
21:	learn: 0.1626616	test: 0.1712905

105:	learn: 0.1598167	test: 0.1706520	best: 0.1706502 (103)	total: 16.8s	remaining: 6.96s
106:	learn: 0.1598054	test: 0.1706527	best: 0.1706502 (103)	total: 17s	remaining: 6.82s
107:	learn: 0.1597800	test: 0.1706480	best: 0.1706480 (107)	total: 17.1s	remaining: 6.67s
108:	learn: 0.1597623	test: 0.1706465	best: 0.1706465 (108)	total: 17.3s	remaining: 6.51s
109:	learn: 0.1597513	test: 0.1706420	best: 0.1706420 (109)	total: 17.5s	remaining: 6.35s
110:	learn: 0.1597328	test: 0.1706400	best: 0.1706400 (110)	total: 17.6s	remaining: 6.17s
111:	learn: 0.1596645	test: 0.1705842	best: 0.1705842 (111)	total: 17.7s	remaining: 6.01s
112:	learn: 0.1595684	test: 0.1705860	best: 0.1705842 (111)	total: 17.9s	remaining: 5.85s
113:	learn: 0.1595373	test: 0.1705862	best: 0.1705842 (111)	total: 18s	remaining: 5.69s
114:	learn: 0.1595147	test: 0.1705793	best: 0.1705793 (114)	total: 18.2s	remaining: 5.53s
115:	learn: 0.1595053	test: 0.1705800	best: 0.1705793 (114)	total: 18.3s	remaining: 5.38s
116:	learn: 0.

{'params': {'depth': 8,
  'l2_leaf_reg': 1,
  'iterations': 150,
  'learning_rate': 0.1},
 'cv_results': defaultdict(list,
             {'iterations': [0,
               1,
               2,
               3,
               4,
               5,
               6,
               7,
               8,
               9,
               10,
               11,
               12,
               13,
               14,
               15,
               16,
               17,
               18,
               19,
               20,
               21,
               22,
               23,
               24,
               25,
               26,
               27,
               28,
               29,
               30,
               31,
               32,
               33,
               34,
               35,
               36,
               37,
               38,
               39,
               40,
               41,
               42,
               43,
               44,
               45,

In [59]:
#saving the model
model.save_model("model_100_150_200_iterations", format="cbm")

In [76]:
#load model
loaded = CatBoostRegressor()
loaded.load_model("model_100_150_200_iterations")

In [62]:
pred_grid = model.predict(X_test)

#calculating MAE
print("MAE: ", mean_absolute_error(y_test, pred_grid))

#calculating R-squared
print("R-squared: ", r2_score(y_test, pred_grid))

MAE:  0.06924709780787434
R-squared:  0.008866324042277385


### Now we try 1 model with loss function MAE, with the params that we obtained from gridsearch

In [38]:
catboost_regressor = CatBoostRegressor(iterations = 150,
                                       learning_rate = 0.1, 
                                       depth = 8, 
                                       loss_function = 'MAE',
                                       cat_features = categorical_indices,
                                       l2_leaf_reg=0.5
                                      )
catboost_regressor.fit(X_train, y_train)

0:	learn: 0.0687555	total: 630ms	remaining: 1m 33s
1:	learn: 0.0686977	total: 1.23s	remaining: 1m 30s
2:	learn: 0.0686425	total: 1.75s	remaining: 1m 25s
3:	learn: 0.0685893	total: 2.34s	remaining: 1m 25s
4:	learn: 0.0685459	total: 2.86s	remaining: 1m 22s
5:	learn: 0.0684997	total: 3.37s	remaining: 1m 20s
6:	learn: 0.0684637	total: 3.88s	remaining: 1m 19s
7:	learn: 0.0684324	total: 4.38s	remaining: 1m 17s
8:	learn: 0.0684006	total: 4.88s	remaining: 1m 16s
9:	learn: 0.0683768	total: 5.39s	remaining: 1m 15s
10:	learn: 0.0683503	total: 5.9s	remaining: 1m 14s
11:	learn: 0.0683148	total: 6.41s	remaining: 1m 13s
12:	learn: 0.0682929	total: 6.92s	remaining: 1m 12s
13:	learn: 0.0682716	total: 7.44s	remaining: 1m 12s
14:	learn: 0.0682461	total: 7.94s	remaining: 1m 11s
15:	learn: 0.0682288	total: 8.45s	remaining: 1m 10s
16:	learn: 0.0682095	total: 8.95s	remaining: 1m 10s
17:	learn: 0.0681866	total: 9.43s	remaining: 1m 9s
18:	learn: 0.0681664	total: 9.92s	remaining: 1m 8s
19:	learn: 0.0681541	tota

In [40]:
pred = catboost_regressor.predict(X_test)

#calculating MAE
print("MAE: ", mean_absolute_error(y_test, pred))

#calculating R-squared
print("R-squared: ", r2_score(y_test, pred))

MAE:  0.06810452451996497
R-squared:  0.006493193731270663


# Trying out catBoost gridsearch 2nd set of params


In [121]:
model2 = CatBoostRegressor(loss_function = 'MAE',
                           eval_metric = 'MAE',
                           cat_features = categorical_indices)

In [122]:
grid2 = {'learning_rate': [0.03, 0.1],
        'depth': [2, 4, 6, 8],
        'l2_leaf_reg': [0.2, 0.5, 1, 3]}

model2.grid_search(grid2, 
                  X=X_train, 
                  y=y_train,
                  cv=3)

0:	learn: 0.0689958	test: 0.0699022	best: 0.0699022 (0)	total: 338ms	remaining: 5m 37s
1:	learn: 0.0689687	test: 0.0698715	best: 0.0698715 (1)	total: 645ms	remaining: 5m 21s
2:	learn: 0.0689439	test: 0.0698453	best: 0.0698453 (2)	total: 890ms	remaining: 4m 55s
3:	learn: 0.0689178	test: 0.0698164	best: 0.0698164 (3)	total: 1.14s	remaining: 4m 44s
4:	learn: 0.0688969	test: 0.0697926	best: 0.0697926 (4)	total: 1.38s	remaining: 4m 35s
5:	learn: 0.0688729	test: 0.0697672	best: 0.0697672 (5)	total: 1.68s	remaining: 4m 37s
6:	learn: 0.0688519	test: 0.0697447	best: 0.0697447 (6)	total: 1.9s	remaining: 4m 29s
7:	learn: 0.0688309	test: 0.0697213	best: 0.0697213 (7)	total: 2.12s	remaining: 4m 22s
8:	learn: 0.0688111	test: 0.0696997	best: 0.0696997 (8)	total: 2.34s	remaining: 4m 17s
9:	learn: 0.0687946	test: 0.0696821	best: 0.0696821 (9)	total: 2.57s	remaining: 4m 14s
10:	learn: 0.0687774	test: 0.0696632	best: 0.0696632 (10)	total: 2.8s	remaining: 4m 11s
11:	learn: 0.0687619	test: 0.0696465	best: 

93:	learn: 0.0683448	test: 0.0691546	best: 0.0691546 (93)	total: 22s	remaining: 3m 31s
94:	learn: 0.0683428	test: 0.0691519	best: 0.0691519 (94)	total: 22.2s	remaining: 3m 31s
95:	learn: 0.0683418	test: 0.0691506	best: 0.0691506 (95)	total: 22.5s	remaining: 3m 31s
96:	learn: 0.0683394	test: 0.0691473	best: 0.0691473 (96)	total: 22.7s	remaining: 3m 31s
97:	learn: 0.0683379	test: 0.0691450	best: 0.0691450 (97)	total: 23s	remaining: 3m 31s
98:	learn: 0.0683361	test: 0.0691427	best: 0.0691427 (98)	total: 23.2s	remaining: 3m 31s
99:	learn: 0.0683339	test: 0.0691400	best: 0.0691400 (99)	total: 23.4s	remaining: 3m 30s
100:	learn: 0.0683322	test: 0.0691384	best: 0.0691384 (100)	total: 23.7s	remaining: 3m 30s
101:	learn: 0.0683301	test: 0.0691363	best: 0.0691363 (101)	total: 24s	remaining: 3m 31s
102:	learn: 0.0683282	test: 0.0691347	best: 0.0691347 (102)	total: 24.2s	remaining: 3m 30s
103:	learn: 0.0683264	test: 0.0691332	best: 0.0691332 (103)	total: 24.5s	remaining: 3m 30s
104:	learn: 0.06832

185:	learn: 0.0681994	test: 0.0689975	best: 0.0689975 (185)	total: 44.8s	remaining: 3m 16s
186:	learn: 0.0681986	test: 0.0689966	best: 0.0689966 (186)	total: 45.2s	remaining: 3m 16s
187:	learn: 0.0681981	test: 0.0689961	best: 0.0689961 (187)	total: 45.5s	remaining: 3m 16s
188:	learn: 0.0681970	test: 0.0689954	best: 0.0689954 (188)	total: 45.7s	remaining: 3m 16s
189:	learn: 0.0681962	test: 0.0689940	best: 0.0689940 (189)	total: 45.9s	remaining: 3m 15s
190:	learn: 0.0681948	test: 0.0689925	best: 0.0689925 (190)	total: 46.1s	remaining: 3m 15s
191:	learn: 0.0681937	test: 0.0689915	best: 0.0689915 (191)	total: 46.3s	remaining: 3m 14s
192:	learn: 0.0681929	test: 0.0689905	best: 0.0689905 (192)	total: 46.6s	remaining: 3m 14s
193:	learn: 0.0681919	test: 0.0689893	best: 0.0689893 (193)	total: 46.8s	remaining: 3m 14s
194:	learn: 0.0681908	test: 0.0689882	best: 0.0689882 (194)	total: 47.1s	remaining: 3m 14s
195:	learn: 0.0681899	test: 0.0689873	best: 0.0689873 (195)	total: 47.4s	remaining: 3m 14s

276:	learn: 0.0681184	test: 0.0689203	best: 0.0689203 (276)	total: 1m 7s	remaining: 2m 55s
277:	learn: 0.0681173	test: 0.0689198	best: 0.0689198 (277)	total: 1m 7s	remaining: 2m 55s
278:	learn: 0.0681166	test: 0.0689196	best: 0.0689196 (278)	total: 1m 7s	remaining: 2m 55s
279:	learn: 0.0681158	test: 0.0689192	best: 0.0689192 (279)	total: 1m 7s	remaining: 2m 54s
280:	learn: 0.0681150	test: 0.0689189	best: 0.0689189 (280)	total: 1m 8s	remaining: 2m 54s
281:	learn: 0.0681142	test: 0.0689182	best: 0.0689182 (281)	total: 1m 8s	remaining: 2m 54s
282:	learn: 0.0681135	test: 0.0689178	best: 0.0689178 (282)	total: 1m 8s	remaining: 2m 54s
283:	learn: 0.0681129	test: 0.0689172	best: 0.0689172 (283)	total: 1m 8s	remaining: 2m 53s
284:	learn: 0.0681122	test: 0.0689168	best: 0.0689168 (284)	total: 1m 9s	remaining: 2m 53s
285:	learn: 0.0681119	test: 0.0689165	best: 0.0689165 (285)	total: 1m 9s	remaining: 2m 53s
286:	learn: 0.0681115	test: 0.0689160	best: 0.0689160 (286)	total: 1m 9s	remaining: 2m 52s

366:	learn: 0.0680650	test: 0.0688742	best: 0.0688742 (366)	total: 1m 29s	remaining: 2m 34s
367:	learn: 0.0680643	test: 0.0688736	best: 0.0688736 (367)	total: 1m 30s	remaining: 2m 34s
368:	learn: 0.0680637	test: 0.0688729	best: 0.0688729 (368)	total: 1m 30s	remaining: 2m 34s
369:	learn: 0.0680631	test: 0.0688724	best: 0.0688724 (369)	total: 1m 30s	remaining: 2m 34s
370:	learn: 0.0680625	test: 0.0688720	best: 0.0688720 (370)	total: 1m 30s	remaining: 2m 33s
371:	learn: 0.0680621	test: 0.0688717	best: 0.0688717 (371)	total: 1m 31s	remaining: 2m 33s
372:	learn: 0.0680613	test: 0.0688710	best: 0.0688710 (372)	total: 1m 31s	remaining: 2m 33s
373:	learn: 0.0680609	test: 0.0688705	best: 0.0688705 (373)	total: 1m 31s	remaining: 2m 33s
374:	learn: 0.0680601	test: 0.0688701	best: 0.0688701 (374)	total: 1m 31s	remaining: 2m 33s
375:	learn: 0.0680595	test: 0.0688697	best: 0.0688697 (375)	total: 1m 32s	remaining: 2m 32s
376:	learn: 0.0680585	test: 0.0688693	best: 0.0688693 (376)	total: 1m 32s	remain

...

275:	learn: 0.0678688	test: 0.0687745	best: 0.0687745 (275)	total: 1m 14s	remaining: 3m 14s
276:	learn: 0.0678685	test: 0.0687745	best: 0.0687745 (275)	total: 1m 14s	remaining: 3m 14s
277:	learn: 0.0678680	test: 0.0687735	best: 0.0687735 (277)	total: 1m 14s	remaining: 3m 13s
278:	learn: 0.0678676	test: 0.0687736	best: 0.0687735 (277)	total: 1m 14s	remaining: 3m 13s
279:	learn: 0.0678676	test: 0.0687736	best: 0.0687735 (277)	total: 1m 15s	remaining: 3m 13s
280:	learn: 0.0678673	test: 0.0687736	best: 0.0687735 (277)	total: 1m 15s	remaining: 3m 13s
281:	learn: 0.0678668	test: 0.0687736	best: 0.0687735 (277)	total: 1m 15s	remaining: 3m 12s
282:	learn: 0.0678649	test: 0.0687729	best: 0.0687729 (282)	total: 1m 16s	remaining: 3m 12s
283:	learn: 0.0678642	test: 0.0687723	best: 0.0687723 (283)	total: 1m 16s	remaining: 3m 12s
284:	learn: 0.0678641	test: 0.0687723	best: 0.0687723 (283)	total: 1m 16s	remaining: 3m 12s
285:	learn: 0.0678635	test: 0.0687719	best: 0.0687719 (285)	total: 1m 16s	remain

365:	learn: 0.0678150	test: 0.0687560	best: 0.0687554 (342)	total: 1m 39s	remaining: 2m 52s
366:	learn: 0.0678148	test: 0.0687558	best: 0.0687554 (342)	total: 1m 39s	remaining: 2m 52s
367:	learn: 0.0678133	test: 0.0687557	best: 0.0687554 (342)	total: 1m 40s	remaining: 2m 51s
368:	learn: 0.0678131	test: 0.0687555	best: 0.0687554 (342)	total: 1m 40s	remaining: 2m 51s
369:	learn: 0.0678121	test: 0.0687544	best: 0.0687544 (369)	total: 1m 40s	remaining: 2m 51s
370:	learn: 0.0678113	test: 0.0687550	best: 0.0687544 (369)	total: 1m 40s	remaining: 2m 51s
371:	learn: 0.0678107	test: 0.0687547	best: 0.0687544 (369)	total: 1m 41s	remaining: 2m 50s
372:	learn: 0.0678106	test: 0.0687545	best: 0.0687544 (369)	total: 1m 41s	remaining: 2m 50s
373:	learn: 0.0678103	test: 0.0687545	best: 0.0687544 (369)	total: 1m 41s	remaining: 2m 50s
374:	learn: 0.0678099	test: 0.0687538	best: 0.0687538 (374)	total: 1m 42s	remaining: 2m 50s
375:	learn: 0.0678080	test: 0.0687546	best: 0.0687538 (374)	total: 1m 42s	remain

455:	learn: 0.0677748	test: 0.0687404	best: 0.0687404 (455)	total: 2m 5s	remaining: 2m 29s
456:	learn: 0.0677744	test: 0.0687409	best: 0.0687404 (455)	total: 2m 5s	remaining: 2m 28s
457:	learn: 0.0677740	test: 0.0687406	best: 0.0687404 (455)	total: 2m 5s	remaining: 2m 28s
458:	learn: 0.0677736	test: 0.0687404	best: 0.0687404 (458)	total: 2m 5s	remaining: 2m 28s
459:	learn: 0.0677735	test: 0.0687406	best: 0.0687404 (458)	total: 2m 6s	remaining: 2m 28s
460:	learn: 0.0677733	test: 0.0687407	best: 0.0687404 (458)	total: 2m 6s	remaining: 2m 27s
461:	learn: 0.0677725	test: 0.0687403	best: 0.0687403 (461)	total: 2m 6s	remaining: 2m 27s
462:	learn: 0.0677719	test: 0.0687396	best: 0.0687396 (462)	total: 2m 7s	remaining: 2m 27s
463:	learn: 0.0677718	test: 0.0687396	best: 0.0687396 (462)	total: 2m 7s	remaining: 2m 27s
464:	learn: 0.0677712	test: 0.0687395	best: 0.0687395 (464)	total: 2m 7s	remaining: 2m 26s
465:	learn: 0.0677711	test: 0.0687396	best: 0.0687395 (464)	total: 2m 8s	remaining: 2m 26s

545:	learn: 0.0677421	test: 0.0687300	best: 0.0687300 (545)	total: 2m 30s	remaining: 2m 5s
546:	learn: 0.0677420	test: 0.0687300	best: 0.0687300 (545)	total: 2m 31s	remaining: 2m 5s
547:	learn: 0.0677419	test: 0.0687297	best: 0.0687297 (547)	total: 2m 31s	remaining: 2m 4s
548:	learn: 0.0677410	test: 0.0687294	best: 0.0687294 (548)	total: 2m 31s	remaining: 2m 4s
549:	learn: 0.0677399	test: 0.0687290	best: 0.0687290 (549)	total: 2m 31s	remaining: 2m 4s
550:	learn: 0.0677393	test: 0.0687283	best: 0.0687283 (550)	total: 2m 32s	remaining: 2m 3s
551:	learn: 0.0677381	test: 0.0687277	best: 0.0687277 (551)	total: 2m 32s	remaining: 2m 3s
552:	learn: 0.0677374	test: 0.0687279	best: 0.0687277 (551)	total: 2m 32s	remaining: 2m 3s
553:	learn: 0.0677371	test: 0.0687281	best: 0.0687277 (551)	total: 2m 32s	remaining: 2m 3s
554:	learn: 0.0677369	test: 0.0687281	best: 0.0687277 (551)	total: 2m 33s	remaining: 2m 2s
555:	learn: 0.0677369	test: 0.0687281	best: 0.0687277 (551)	total: 2m 33s	remaining: 2m 2s

KeyboardInterrupt: 

---
## Extra notes

- ~perform CV~
- Scaling of numerical features


- instead of constant lr, can use optimiser
- maybe can try cleaning data before training (e.g removing those col with 1 single value)
- have to play around with the hyperparams for catBoost
- can try ensemble learning (combining the predictions from multiple models)
- can try 2017 jan just starting from 1 instead of 13
- obtaining accuracy and error values from the model so that we can plot and observe 
- can try label encoding (same as jr)